In [ ]:
from funcs import load_matrices, matrix_funcs
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import riccati_jn, riccati_yn
from airy_props import *
import os


dirs = [n for n in os.listdir() if (os.path.isdir(n) and ((n[-2:]=='_p') or (n[-2:]=='_s')))]
# dirs = ['H_1_s', 'H_0_plus_s', 'H_0_minus_s', 'H_2_s', 'H_3_p']
for dir in dirs:
    r, hls, ddrls = load_matrices(dir)
    print(dir)
    emin = hls[-1,0,0]
    n = len(hls[0])
    shift = np.array([emin*np.eye(len(hls[0]), dtype=complex)]*len(r))
    vd = (hls-shift)/4.637

    hfunc, ddrfunc = matrix_funcs(r, vd, ddrls)
    emax = np.abs(np.array(hfunc(r[0])[-1,-1]))

    w = lambda rs, l, e: np.array([e*np.eye(n)-(hfunc(r)+l*(l*1)/(r**2)*np.eye(n)) for r in rs])
    wp = lambda rs, l: np.array([-(hfunc(r,1)-2*l*(l*1)/(r**3)*np.eye(n)) for r in rs])

    es = np.array([1])
    smats = {}
    for i in range(n):
        emax = np.abs(np.array(hfunc(r[0])[i,i]))
        emin = np.abs(np.array(hfunc(r[-1])[i,i]))
        es = np.concatenate((es, np.power(10, np.arange(np.log10(emin if emin!=0 else 1), np.log10(emax), 0.01))))
    es = np.unique(np.round(np.sort(es)))
    sigmas = np.zeros((len(es), n, n))
    ecm = es*4.637+np.real(hls[-1,0,0])
    for ei in range(len(es)):
        e = es[ei]
        lmax = int(np.round(np.sqrt(e)*np.max(r[-1]))*0.8)
        ks = np.sqrt(np.abs(np.diag(e*np.eye(n)-hfunc(r[-1]))))
        yf = np.zeros((lmax, n, n), dtype=complex)
        nt = np.sum(np.diag(vd[-1]) < e)
        for l in range(lmax):
           yf[l] = airy_imb_prop(r, w(r[:-1]+np.diff(r)/2, l, e), wp(r[:-1]+np.diff(r)/2, l), n)
        ytot = np.array([(yf[l]+np.matrix(yf[l]).H)/2 for l in range(lmax)])
        jmat = np.zeros((lmax, n, n))
        nmat = np.zeros((lmax, n, n))
        jpmat = np.zeros((lmax, n, n))
        npmat = np.zeros((lmax, n, n))
        for j in range(n):
            if j <= nt:
                prej, prejp = riccati_jn(lmax-1, np.real(ks[j])*r[-1])
                pren, prenp = riccati_yn(lmax-1, np.real(ks[j])*r[-1])
                jmat[:,j,j] = prej/np.sqrt(np.real(ks[j]))
                nmat[:,j,j] = pren/np.sqrt(np.real(ks[j]))
                jpmat[:,j,j] = np.sqrt(np.real(ks[j])) * prejp
                npmat[:,j,j] = np.sqrt(np.real(ks[j])) * prenp
            else:
                jmat[:,j,j], prejp = riccati_in(lmax-1, np.real(ks[j])*r[-1])
                nmat[:,j,j], prenp = riccati_kn(lmax-1, np.real(ks[j])*r[-1])
                jpmat[:,j,j] = ks[j] * prejp
                npmat[:,j,j] = ks[j] * prenp
        rmat = np.array([-np.matrix(npmat[l] - ytot[l] @ nmat[l]).I @ np.matrix(jpmat[l] - ytot[l] @ jmat[l]) for l in range(lmax)])
        smat = np.array([np.matrix(np.eye(nt) + 1j*rmat[l,:nt,:nt]).I @ np.matrix(np.eye(nt) - 1j*rmat[l,:nt,:nt]) if not np.all(np.isnan(rmat[l,:nt,:nt])) else np.eye(nt) for l in range(lmax)])
        print(smat[0])
        smats[f'E = {ecm[ei]}'] = smat
    np.savez(f'{dir}/Smatrix.npz', smats)
    print(f'Directory {dir} is done!')

/home/andrey/projects/ArHe_Inelastic/funcs.py:56: SyntaxWarning: invalid escape sequence '\O'
  spatchom = [mlines.Line2D([], [], color='black', linestyle=lines[k], label=f'$\Omega = {k}$') for k in lines.keys() if k != 'nist' and int(k[0]) < 3]
/home/andrey/projects/ArHe_Inelastic/funcs.py:59: SyntaxWarning: invalid escape sequence '\O'
  ppatchom = [mlines.Line2D([], [], color='black', linestyle=lines[k], label=f'$\Omega = {k}$') for k in lines.keys() if k != 'nist']
/home/andrey/projects/ArHe_Inelastic/funcs.py:383: SyntaxWarning: invalid escape sequence '\A'
  plt.xlabel('R, \AA')
/home/andrey/projects/ArHe_Inelastic/funcs.py:401: SyntaxWarning: invalid escape sequence '\R'
  signre = f'$\Re V_{{{signs[i][:2]}_{{{signs[i][2:]}}}, {signs[j][:2]}_{{{signs[j][2:]}}}}}$'
/home/andrey/projects/ArHe_Inelastic/funcs.py:402: SyntaxWarning: invalid escape sequence '\I'
  signim = f'$\Im V_{{{signs[i][:2]}_{{{signs[i][2:]}}}, {signs[j][:2]}_{{{signs[j][2:]}}}}}$'
/home/andrey/projects/ArHe_I

H_1_s
[[-0.81892703+0.57389765j]]
[[-0.42334452-0.90596877j]]
[[0.52923734+0.84847383j]]
[[-0.0588288-0.99826809j]]
[[-0.73138067+0.68196944j]]
[[0.92201398+0.3871566j]]
[[0.17797402-0.98403519j]]
[[-0.99316195-0.11674476j]]
[[-0.18499947+0.98273862j]]
[[0.9354291+0.35351436j]]
[[0.58521204-0.81088031j]]
[[-0.56866855-0.82256676j]]
[[-0.98021638+0.19792892j]]
[[-0.25925533+0.96580882j]]
[[0.71479709+0.69933191j]]
[[0.97179232-0.23583825j]]
[[0.35392218-0.93527487j]]
[[-0.54037908-0.84142168j]]
[[-0.99473783-0.10245317j]]
[[-0.72318101+0.69065855j]]
[[0.02716064+0.99963108j]]
[[0.74166888+0.67076619j]]
[[0.99883551-0.04824547j]]
[[0.69017678-0.72364081j]]
[[0.02203763-0.99975714j]]
[[-0.64241521-0.76635677j]]
[[-0.98517735-0.1715389j]]
[[-0.87244045+0.48872044j]]
[[-0.38385721+0.92339246j]]
[[0.25318945+0.96741672j]]
[[0.77716844+0.62929263j]]
[[0.99811211+0.06141833j]]
[[0.85577696-0.51734495j]]
[[0.42131354-0.90691505j]]
[[-0.14442377-0.98951593j]]
[[-0.65411004-0.7563994j]]
[[-0.9550